## *Import libraries*

In [27]:
import re
import os
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tech\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tech\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## *Loading dataset*

In [28]:
data = pd.read_csv('Arabic_poetry_dataset.csv')

# Take a random sample from the dataset
df = data.sample(n=10000, random_state=42)

# Print the sample
df

,id,category,poet_name,poem_title,poem_text
16765,16939,العصر العباسي,أبو العلاء المعري,سكنت إلى الدنيا فلما عرفتها,سَكَنتُ إِلى الدُنيا فَلَمّا عَرَفتُها\nتَمَنّ...
11400,11574,مصر,مصطفى صادق الرافعي,آفة العالم أن لا يعملا,آفةُ العالمِ أن لا يعملا\nوشقا الجاهلِ أن لا ي...
37613,37787,عصر المخضرمون,علي بن أبي طالب,أبى الله إلا أن صفين دارنا,أَبى اللَهُ إِلّا أَنَّ صِفِّينَ دارُنا\nوَدار...
13700,13874,العصر الجاهلي,الخنساء,يا عين جودي بالدموع,يا عَينِ جودي بِالدُموعِ \nالمُستَهِلّاتِ السَ...
3272,3379,اليمن,محمد الشوكاني,كأن وجه النهر إذ حفت به,كَأَنَّ وَجْهَ النَّهْرِ إذْ حَفَّتْ بِهِ\nأَش...
...,...,...,...,...,...
2206,2313,المغرب,أبو الفيض الكتاني,بهيج لي العهد القديم صبابة,بهيج لي العهد القديم صبابة\nأنوح بها نوح الحما...
50646,50820,العصر المملوكي,لسان الدين بن الخطيب,نادتني الأيام عند لقائه,نَادَتْنِيَ الأَيَّامُ عِنْدَ لِقَائِهِ\nوَهْي...
23805,23979,العصر العباسي,أبو الفتح البستي,أرى جلنارا قلوب الورى,أرى جُلّناراً قُلوبَ الوَرى\nلِما فَوقَ خَدَّي...
33433,33607,العصر العثماني,ابن معصوم,هجر الحبائب جانبي,هجرَ الحبائبُ جانبي\nونزلنَ مُنعرَجَ اللوى\nفَ...


In [67]:
df.describe()

,id,cosine_similarity
count,10000.000000,10000.000000
mean,27541.408900,0.000166
std,15866.083131,0.005501
min,4.000000,0.000000
25%,13816.500000,0.000000
50%,27611.000000,0.000000
75%,41153.250000,0.000000
max,55114.000000,0.353553


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 16765 to 21913
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   category           10000 non-null  object 
 2   poet_name          10000 non-null  object 
 3   poem_title         10000 non-null  object 
 4   poem_text          10000 non-null  object 
 5   cleaned_poems      10000 non-null  object 
 6   lemmatize_text     10000 non-null  object 
 7   stemming_text      10000 non-null  object 
 8   cosine_similarity  10000 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 781.2+ KB


In [66]:
df.columns

Index(['id', 'category', 'poet_name', 'poem_title', 'poem_text',
       'cleaned_poems', 'lemmatize_text', 'stemming_text',
       'cosine_similarity'],
      dtype='object')

## *Preprocssing Dataset*

In [29]:
def preprocess_doc_lemma(doc):
    
    #normalization
    doc = re.sub("[إأآا]", "ا", doc)
    doc = re.sub("ى", "ي", doc)
    doc = re.sub("ؤ", "ء", doc)
    doc = re.sub("ئ", "ء", doc)
    doc = re.sub("ة", "ه", doc)
    doc = re.sub("گ", "ك", doc)
    doc = re.sub(r'[^\w\s]', '', doc)
    
    # Tokenization
    tokens = word_tokenize(doc)
    
    # Stop-word removal
    stop_words = set(nltk.corpus.stopwords.words("arabic"))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    filtered_sentences=" ".join(lemmatized_tokens)
    return filtered_sentences

In [30]:
df['cleaned_poems'] = df['poem_text'].apply(preprocess_doc_lemma)
df

,id,category,poet_name,poem_title,poem_text,cleaned_poems
16765,16939,العصر العباسي,أبو العلاء المعري,سكنت إلى الدنيا فلما عرفتها,سَكَنتُ إِلى الدُنيا فَلَمّا عَرَفتُها\nتَمَنّ...,سكنت الي الدنيا فلما عرفتها تمنيت اني بساكن فت...
11400,11574,مصر,مصطفى صادق الرافعي,آفة العالم أن لا يعملا,آفةُ العالمِ أن لا يعملا\nوشقا الجاهلِ أن لا ي...,افه العالم ان يعملا وشقا الجاهل ان يسالا انما ...
37613,37787,عصر المخضرمون,علي بن أبي طالب,أبى الله إلا أن صفين دارنا,أَبى اللَهُ إِلّا أَنَّ صِفِّينَ دارُنا\nوَدار...,ابي الله الا ان صفين دارنا وداركم لاح الافق كو...
13700,13874,العصر الجاهلي,الخنساء,يا عين جودي بالدموع,يا عَينِ جودي بِالدُموعِ \nالمُستَهِلّاتِ السَ...,جودي بالدموع المستهلات السواجم فيضا انخرق الجم...
3272,3379,اليمن,محمد الشوكاني,كأن وجه النهر إذ حفت به,كَأَنَّ وَجْهَ النَّهْرِ إذْ حَفَّتْ بِهِ\nأَش...,وجه النهر اذ حفت اشجاره فصافحته الاغصن مراه غي...
...,...,...,...,...,...,...
2206,2313,المغرب,أبو الفيض الكتاني,بهيج لي العهد القديم صبابة,بهيج لي العهد القديم صبابة\nأنوح بها نوح الحما...,بهيج العهد القديم صبابه انوح نوح الحمام علي اي...
50646,50820,العصر المملوكي,لسان الدين بن الخطيب,نادتني الأيام عند لقائه,نَادَتْنِيَ الأَيَّامُ عِنْدَ لِقَائِهِ\nوَهْي...,نادتني الايام لقاءه وهي تغفل التنبيها ابن الخط...
23805,23979,العصر العباسي,أبو الفتح البستي,أرى جلنارا قلوب الورى,أرى جُلّناراً قُلوبَ الوَرى\nلِما فَوقَ خَدَّي...,اري جلنارا قلوب الوري خديه جلنار
33433,33607,العصر العثماني,ابن معصوم,هجر الحبائب جانبي,هجرَ الحبائبُ جانبي\nونزلنَ مُنعرَجَ اللوى\nفَ...,هجر الحباءب جانبي ونزلن منعرج اللوي فظللت اعسف...


## *Query Preprocessing*

In [31]:
def normalize_arabic(text):
    if not isinstance(text, (str, bytes)):
        # If input is not a string or bytes-like object, convert it to string
        text = str(text)
        
    if text.startswith("ال"):
        text = text[2:]
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("[ًٌٍَُِّْ]", "", text)
    return text

def preprocess_arabic_Remove_stop(words):
    stop_words = set(stopwords.words('arabic'))
    preprocessed_text=[]
    words = [word for word in words if word not in stop_words]
    preprocessed_text = words
    return preprocessed_text

def remove_punctuation(words):
    cleaned_words = [word for word in words if word not in string.punctuation]
    return cleaned_words

def lemma(words):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

def stemming(text):
    stemmer = PorterStemmer()
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

def preprocess_text(text):
    words = word_tokenize(text)
    out = remove_punctuation(words)
    out = preprocess_arabic_Remove_stop(out)
    out = lemma(out)
    return out

df['lemmatize_text'] = df['poem_text'].apply(lemma)
df['stemming_text'] = df['poem_text'].apply(stemming)

In [32]:
df

,id,category,poet_name,poem_title,poem_text,cleaned_poems,lemmatize_text,stemming_text
16765,16939,العصر العباسي,أبو العلاء المعري,سكنت إلى الدنيا فلما عرفتها,سَكَنتُ إِلى الدُنيا فَلَمّا عَرَفتُها\nتَمَنّ...,سكنت الي الدنيا فلما عرفتها تمنيت اني بساكن فت...,"[س, َ, ك, َ, ن, ت, ُ, , إ, ِ, ل, ى, , ا, ل, ...",سَكَنتُ إِلى الدُنيا فَلَمّا عَرَفتُها تَمَنَّ...
11400,11574,مصر,مصطفى صادق الرافعي,آفة العالم أن لا يعملا,آفةُ العالمِ أن لا يعملا\nوشقا الجاهلِ أن لا ي...,افه العالم ان يعملا وشقا الجاهل ان يسالا انما ...,"[آ, ف, ة, ُ, , ا, ل, ع, ا, ل, م, ِ, , أ, ن, ...",آفةُ العالمِ أن لا يعملا وشقا الجاهلِ أن لا يس...
37613,37787,عصر المخضرمون,علي بن أبي طالب,أبى الله إلا أن صفين دارنا,أَبى اللَهُ إِلّا أَنَّ صِفِّينَ دارُنا\nوَدار...,ابي الله الا ان صفين دارنا وداركم لاح الافق كو...,"[أ, َ, ب, ى, , ا, ل, ل, َ, ه, ُ, , إ, ِ, ل, ...",أَبى اللَهُ إِلّا أَنَّ صِفِّينَ دارُنا وَدارُ...
13700,13874,العصر الجاهلي,الخنساء,يا عين جودي بالدموع,يا عَينِ جودي بِالدُموعِ \nالمُستَهِلّاتِ السَ...,جودي بالدموع المستهلات السواجم فيضا انخرق الجم...,"[ي, ا, , ع, َ, ي, ن, ِ, , ج, و, د, ي, , ب, ...",يا عَينِ جودي بِالدُموعِ المُستَهِلّاتِ السَوا...
3272,3379,اليمن,محمد الشوكاني,كأن وجه النهر إذ حفت به,كَأَنَّ وَجْهَ النَّهْرِ إذْ حَفَّتْ بِهِ\nأَش...,وجه النهر اذ حفت اشجاره فصافحته الاغصن مراه غي...,"[ك, َ, أ, َ, ن, ّ, َ, , و, َ, ج, ْ, ه, َ, , ...",كَأَنَّ وَجْهَ النَّهْرِ إذْ حَفَّتْ بِهِ أَشْ...
...,...,...,...,...,...,...,...,...
2206,2313,المغرب,أبو الفيض الكتاني,بهيج لي العهد القديم صبابة,بهيج لي العهد القديم صبابة\nأنوح بها نوح الحما...,بهيج العهد القديم صبابه انوح نوح الحمام علي اي...,"[ب, ه, ي, ج, , ل, ي, , ا, ل, ع, ه, د, , ا, ...",بهيج لي العهد القديم صبابة أنوح بها نوح الحمام...
50646,50820,العصر المملوكي,لسان الدين بن الخطيب,نادتني الأيام عند لقائه,نَادَتْنِيَ الأَيَّامُ عِنْدَ لِقَائِهِ\nوَهْي...,نادتني الايام لقاءه وهي تغفل التنبيها ابن الخط...,"[ن, َ, ا, د, َ, ت, ْ, ن, ِ, ي, َ, , ا, ل, أ, ...",نَادَتْنِيَ الأَيَّامُ عِنْدَ لِقَائِهِ وَهْيَ...
23805,23979,العصر العباسي,أبو الفتح البستي,أرى جلنارا قلوب الورى,أرى جُلّناراً قُلوبَ الوَرى\nلِما فَوقَ خَدَّي...,اري جلنارا قلوب الوري خديه جلنار,"[أ, ر, ى, , ج, ُ, ل, ّ, ن, ا, ر, ا, ً, , ق, ...",أرى جُلّناراً قُلوبَ الوَرى لِما فَوقَ خَدَّيْ...
33433,33607,العصر العثماني,ابن معصوم,هجر الحبائب جانبي,هجرَ الحبائبُ جانبي\nونزلنَ مُنعرَجَ اللوى\nفَ...,هجر الحباءب جانبي ونزلن منعرج اللوي فظللت اعسف...,"[ه, ج, ر, َ, , ا, ل, ح, ب, ا, ئ, ب, ُ, , ج, ...",هجرَ الحبائبُ جانبي ونزلنَ مُنعرَجَ اللوى فَظل...


## *Inverted index*

In [33]:
import string
def bulid_inverted_index(documents):
    inverted_index = dict()
    for doc_id, doc in enumerate(documents):
        doc = normalize_arabic(doc)
        terms = preprocess_text(doc)
        for term in terms:
            if term not in inverted_index:
                inverted_index[term] = []
            if doc_id not in inverted_index[term]:
                inverted_index[term].append(doc_id)
    return inverted_index
inverted_index = bulid_inverted_index(df['cleaned_poems'])

## *Searching*

In [34]:
def search(query, inverted_index, documents):
    query = normalize_arabic(query)
    query = preprocess_text (query)
    relevant_doc_ids = list(range(len(documents)))
    #print(query)
    for term in query:
        relevant_docs = []
        if term in inverted_index:
            p1 = 0
            p2 = 0
            while p1 < len(relevant_doc_ids) and p2 < len(inverted_index[term]):
                if relevant_doc_ids[p1] < inverted_index[term][p2]:
                    p1 += 1
                elif relevant_doc_ids[p1] > inverted_index[term][p2]:
                    p2 += 1
                else:
                    relevant_docs.append(relevant_doc_ids[p1])
                    p1 += 1
                    p2 += 1
            relevant_doc_ids = relevant_docs
            result = df.iloc[relevant_doc_ids]

    return result.iloc[:,0]

## *Ranking*

In [69]:
def Ranking(query):
    vectorizer = CountVectorizer()

    # Fit and transform the text data
    X = vectorizer.fit_transform(df['cleaned_poems'])

    # Tokenize and vectorize the query
    query_vector = vectorizer.transform([query])

    query_vector_dense = query_vector.toarray()

    # Calculate cosine similarity between query and documents
    cosine_similarities = cosine_similarity(query_vector_dense, X)

    # Add cosine similarities to DataFrame
    df['cosine_similarity'] = cosine_similarities[0]

    # Sort DataFrame by cosine similarity
    df_sorted = df.sort_values(by='cosine_similarity', ascending=False)

    # Filter out documents with cosine similarity greater than 0
    similar_documents = df_sorted[df_sorted['cosine_similarity'] > 0.3]

    #similar_document_ids = similar_documents.iloc[:, 0]

    return similar_documents[['cleaned_poems','cosine_similarity']]

In [70]:
Query="العالم"

In [71]:
search(Query,inverted_index,df['cleaned_poems'])

4243      4350
51235    51409
6268      6405
28722    28896
4021      4128
         ...  
816        923
34604    34778
38584    38758
54004    54178
50566    50740
Name: id, Length: 142, dtype: int64

In [72]:
cosine_similarities = Ranking(Query)
type(cosine_similarities)

pandas.core.frame.DataFrame

## *Evaluation*

In [73]:
retrieved_documents_list = [1 if doc_id in cosine_similarities.iloc[:,0] else 0 for doc_id in range(len(df))]
print(retrieved_documents_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [74]:
def calculate_relevant_documents(query, inverted_index, documents):
    query = preprocess_text(query)
    relevant_doc_ids = set(range(len(documents)))

    for term in query:
        if term in inverted_index:
            relevant_doc_ids &= set(inverted_index[term])

    return relevant_doc_ids

query = 'العالم'
relevant_documents = calculate_relevant_documents(query, inverted_index, df['cleaned_poems'])
print(relevant_documents)


{1, 7682, 2052, 7, 531, 7190, 7194, 2080, 6183, 7726, 1595, 9803, 8784, 3673, 97, 5738, 2164, 6280, 8843, 2705, 7321, 1190, 6311, 1727, 3266, 1219, 3273, 4299, 5836, 4818, 9944, 737, 4323, 2279, 4344, 1795, 4366, 9486, 1815, 793, 6446, 3378, 2355, 5945, 314, 5949, 4932, 5444, 2886, 4937, 6477, 4435, 9561, 866, 7011, 2918, 359, 6506, 4981, 9080, 6529, 8587, 9614, 6543, 2969, 2465, 6568, 9652, 1986, 8653, 6095, 9686, 5082, 5595, 7646, 9701, 1002, 8173, 5104, 4597, 8186, 3068}


In [75]:
ground_truth = [1 if doc_id in relevant_documents else 0 for doc_id in range(len(df))]
print(ground_truth)

[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [76]:
def precision_score(ground_truth, retrieved_documents):
    # Calculate True Positives (TP) and False Positives (FP)
    TP = sum(1 for gt, retrieved in zip(ground_truth, retrieved_documents) if gt == 1 and retrieved == 1)
    FP = sum(1 for gt, retrieved in zip(ground_truth, retrieved_documents) if gt == 0 and retrieved == 1)

    # Calculate Precision
    if TP + FP == 0:
        return 0
    else:
        return TP / (TP + FP)

def recall_score(ground_truth, retrieved_documents):
    # Calculate True Positives (TP) and False Negatives (FN)
    TP = sum(1 for gt, retrieved in zip(ground_truth, retrieved_documents) if gt == 1 and retrieved == 1)
    FN = sum(1 for gt, retrieved in zip(ground_truth, retrieved_documents) if gt == 1 and retrieved == 0)

    # Calculate Recall
    if TP + FN == 0:
        return 0
    else:
        return TP / (TP + FN)

def f1_score(ground_truth, retrieved_documents):
    precision = precision_score(ground_truth, retrieved_documents)
    recall = recall_score(ground_truth, retrieved_documents)

    # Calculate F1 Score
    if precision + recall == 0:
        return 0
    else:
        return 2 * (precision * recall) / (precision + recall)

In [77]:
# Calculate Precision, Recall, and F1 Score
precision = precision_score(ground_truth, retrieved_documents_list)
recall = recall_score(ground_truth, retrieved_documents_list)
f1 = f1_score(ground_truth, retrieved_documents_list)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0
Recall: 0.0
F1 Score: 0


## *GUI*

In [81]:
import tkinter as tk

def indexing():
    result_text.delete(1.0, tk.END)  # Clear previous results
    query_text = query_entry.get()
    search_text = search(query_text, inverted_index, df)
    #positional_index = build_positional_index(query_text)
    result_text.insert(tk.END, search_text)
    

def ranking():
    result_text.delete(1.0, tk.END)  # Clear previous results
    query_text = query_entry.get()
    rank = Ranking(query_text)
    result_text.insert(tk.END, rank)
    
    
def exit_app():
    root.destroy()

# Create the main window
root = tk.Tk()
root.title("Search Application")

# Change the background color of the window
root.configure(bg="#17202A")

# Set window size and position
window_width = 800
window_height = 600
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_coordinate = (screen_width / 2) - (window_width / 2)
y_coordinate = (screen_height / 2) - (window_height / 2)
root.geometry("%dx%d+%d+%d" % (window_width, window_height, x_coordinate, y_coordinate))

# Create and place the widgets
query_label = tk.Label(root, text="Enter your query:", font=("Arial", 24, "bold"), bg="#17202A", fg="white")
query_label.pack(pady=20)

query_entry = tk.Entry(root, font=("Arial", 16), width=40)
query_entry.pack(pady=10)

button_frame = tk.Frame(root, bg="#17202A")
button_frame.pack(pady=20)

index_button = tk.Button(button_frame, text="Indexing", command=indexing, font=("Arial", 14), bg="#3498db", fg="white", width=15)
index_button.grid(row=0, column=0, padx=10)

rank_button = tk.Button(button_frame, text="Ranking", command=ranking, font=("Arial", 14), bg="#3498db", fg="white", width=15)
rank_button.grid(row=0, column=1, padx=10)

result_label = tk.Label(root, text="Results:", font=("Arial", 18, "bold"), bg="#17202A", fg="#e74c3c")
result_label.pack(pady=20)

# Larger text field
result_text = tk.Text(root, height=10, width=60, font=("Arial", 12), bg="#ffffff", fg="#333333")
result_text.pack(pady=10)

exit_button = tk.Button(root, text="Exit", command=exit_app, font=("Arial", 14), bg="#e74c3c", fg="white", width=10)
exit_button.pack(pady=20)

# Start the GUI event loop
root.mainloop()
